In [5]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D

In [6]:
img_width, img_height = 256, 256

### Build the network 
img_input = Input(shape=(256, 256, 3))
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

model = Model(input = img_input, output = x)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
Total para

/home/himani/.local/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  


In [20]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
[layer.name for layer in model.layers]


['input_2',
 'block1_conv1',
 'block1_conv2',
 'block1_pool',
 'block2_conv1',
 'block2_conv2',
 'block2_pool']

In [40]:
import h5py
#weights_path = 'vgg19_weights.h5' # ('https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5)
weights_path = '/home/himani/vgg19_weights_tf_dim_ordering_tf_kernels.h5'
f = h5py.File(weights_path)
model.load_weights(weights_path, by_name=True)
for key in f.keys(): print(key)

model.load_weights(weights_path, by_name=True)
layer_count=0;
for layer in model.layers:
    weights = layer.get_weights()
    layer_count=layer_count+1
    print("[INFO] Model Layer Configuration with respect to each layer weights as follows : " + str(layer.get_config()), str(layer.name),str(weights))
print("[INFO] The total number layers is : " + str(layer_count))
layer.index("block1_conv1")
for i in layer_dict.keys():
    index = layer.index(i)
    model.layers[index].set_weights(weights)




block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool
fc1
fc2
flatten
predictions
[INFO] Model Layer Configuration with respect to each layer weights as follows : {'sparse': False, 'dtype': 'float32', 'name': 'input_2', 'batch_input_shape': (None, 256, 256, 3)} input_2 []
[INFO] Model Layer Configuration with respect to each layer weights as follows : {'kernel_initializer': {'config': {'seed': None, 'mode': 'fan_avg', 'distribution': 'uniform', 'scale': 1.0}, 'class_name': 'VarianceScaling'}, 'filters': 64, 'padding': 'same', 'kernel_regularizer': None, 'bias_constraint': None, 'kernel_size': (3, 3), 'bias_regularizer': None, 'use_bias': True, 'activity_regularizer': None, 'name': 'block1_conv1', 'activation': 'relu', 'data_format': 'channels_last', 'kernel_constraint': None,

AttributeError: 'MaxPooling2D' object has no attribute 'index'

In [31]:
layer_names = [layer.name for layer in model.layers]

In [38]:
f["block1_conv1"].attrs["weight_names"]
array([b'block1_conv1/kernel:0', b'block1_conv1/bias:0'], 
      dtype='|S21')

NameError: name 'array' is not defined

In [10]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/home/himani/MobileNet/profile',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',shuffle=True)

Found 4451 images belonging to 2 classes.


In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,epochs=10)

Epoch 1/10
139/139 [==============================] - 556s 4s/step - loss: 0.1553 - acc: 0.9780
Epoch 2/10
115/139 [=======================>......] - ETA: 1:36 - loss: 0.3489 - acc: 0.9784